# 서울시 전기차 충전 데이터 이용

In [9]:
import pandas as pd
df = pd.read_csv(r'D:\charging_data\charging_data_modify.csv', encoding='cp949')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10972 entries, 0 to 10971
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   순번        10972 non-null  int64  
 1   충전소명      10972 non-null  object 
 2   충전일       10972 non-null  object 
 3   충전시작시간    10972 non-null  object 
 4   충전종료시간    10972 non-null  object 
 5   충전량(kWh)  10972 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 514.4+ KB
None


In [10]:
# 쓸모없는 열 삭제
df = df.drop('순번', axis=1)
df = df.drop('충전량', axis=1)


# 충전소명을 기준으로 데이터 수 확인 
df['충전소명'].value_counts()

충전소명
양재솔라스테이션        2783
동작구청            1211
양천구청             960
홍제1동 공용주차장       688
어린이대공원 후문주차장     651
중랑구청             618
성동구청             537
남부초 공영주차장        476
서대문구청 주차장        449
서소문청사(지상 급속)     265
망우제2 거주자주차장      265
복정역 공영주차장        225
송월동차고지(급속)       218
송월동차고지(완속)       209
마곡중앙광장공영주차장      209
개화산역 공영주차장       183
서울시 본관청사(완속)     173
가산 공영주차장         153
남산청사             153
서소문청사(지하 급속)     138
창동역 공영주차장        111
서소문청사(후생동)        87
서울시 본관청사(급속)      79
서소문청사(완속)         62
천호역 공영주차장         62
중화1동 공영주차장         7
Name: count, dtype: int64

In [27]:
df['충전일자'] = pd.to_datetime(df['충전일'])
df['충전일자'].min() 

Timestamp('2021-06-16 00:00:00')

In [11]:
df['충전시작시간'] = pd.to_datetime(df['충전시작시간'])
df['충전종료시간'] = pd.to_datetime(df['충전종료시간'])

df['충전시간'] = (df['충전종료시간'] - df['충전시작시간']).dt.total_seconds() // 60
df['충전시간']


0        27.0
1        40.0
2        15.0
3        23.0
4        40.0
         ... 
10967    29.0
10968    53.0
10969    42.0
10970    53.0
10971    33.0
Name: 충전시간, Length: 10972, dtype: float64

In [12]:
df = df[df['충전시간'] >= 5]
df = df[df['충전시간'] <= 180]

df['충전소명'].value_counts()

충전소명
양재솔라스테이션        2669
동작구청            1192
양천구청             938
홍제1동 공용주차장       663
어린이대공원 후문주차장     626
중랑구청             593
성동구청             527
남부초 공영주차장        463
서대문구청 주차장        423
서소문청사(지상 급속)     256
망우제2 거주자주차장      252
송월동차고지(급속)       209
복정역 공영주차장        207
개화산역 공영주차장       175
마곡중앙광장공영주차장      148
가산 공영주차장         142
남산청사             139
서울시 본관청사(완속)     134
서소문청사(지하 급속)     132
송월동차고지(완속)       119
창동역 공영주차장        104
서울시 본관청사(급속)      70
천호역 공영주차장         57
서소문청사(완속)         51
서소문청사(후생동)        33
중화1동 공영주차장         4
Name: count, dtype: int64

In [13]:
# 충전 세션 500개 이하 삭제
filter = df['충전소명'].value_counts()

# 충전 세션이 500개 이하인 경우 삭제
to_drop = filter[filter <= 500].index

# to_drop에 들어있지 않은 loc만 추출
df = df[~df['충전소명'].isin(to_drop)]

df['충전소명'].value_counts()

충전소명
양재솔라스테이션        2669
동작구청            1192
양천구청             938
홍제1동 공용주차장       663
어린이대공원 후문주차장     626
중랑구청             593
성동구청             527
Name: count, dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7191 entries, 965 to 10971
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   충전소명      7191 non-null   object        
 1   충전일       7191 non-null   object        
 2   충전시작시간    7191 non-null   datetime64[ns]
 3   충전종료시간    7191 non-null   datetime64[ns]
 4   충전량(kWh)  7191 non-null   float64       
 5   충전시간      7191 non-null   float64       
dtypes: datetime64[ns](2), float64(2), object(2)
memory usage: 393.3+ KB


In [14]:
# 요일을 0-월 1-화 ... 6-일 로 맵핑 (datetime 객체 이용)

df['요일'] = pd.to_datetime(df['충전시작시간'])

df['요일'] = df['충전시작시간'].dt.weekday

df['요일'].value_counts()

요일
4    1192
3    1067
5    1053
2    1013
0    1002
1     995
6     886
Name: count, dtype: int64

In [17]:
# 주말인지 아닌지 여부 데이터 추가

def is_weekend(day):
    if day in [0,1,2,3,4]: # 0: 월 6: 일
        return 0
    else:
        return 1

df['주말'] = df['요일'].apply(is_weekend)
df['주말'].value_counts()


주말
0    5269
1    1939
Name: count, dtype: int64

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7208 entries, 965 to 10971
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   충전소명      7208 non-null   object        
 1   충전일       7208 non-null   object        
 2   충전시작시간    7208 non-null   datetime64[ns]
 3   충전종료시간    7208 non-null   datetime64[ns]
 4   충전량(kWh)  7208 non-null   float64       
 5   충전시간      7208 non-null   float64       
 6   요일        7208 non-null   int32         
 7   주말        7208 non-null   int64         
dtypes: datetime64[ns](2), float64(2), int32(1), int64(1), object(2)
memory usage: 478.7+ KB


새로운 데이터프레임 객체 생성

In [25]:
df_2 = pd.DataFrame()
df_2['time'] = [i for i in range(144)] * 117
df_2['time'].tail()

16843    139
16844    140
16845    141
16846    142
16847    143
Name: time, dtype: int64

In [36]:
# 시작 날짜(2021년 6월 16일 기준)으로 144개씩 데이터 구분 10분 주기 -> 10T
date_range = pd.date_range('2021-06-16', periods=len(df_2['time']), freq='10T') 

# 요일을 리턴
day_of_week = date_range.dayofweek

# 요일에 기반하여 5 이상인 경우 주말(1), 아닌경우 (0)
df_2['weekend'] = (day_of_week >= 5).astype(int) # 1 : 주말 0: 평일
df_2['dayofweek'] = day_of_week

df_2.head(160)

,time,weekend,dayofweek
0,0,0,2
1,1,0,2
2,2,0,2
3,3,0,2
4,4,0,2
...,...,...,...
155,11,0,3
156,12,0,3
157,13,0,3
158,14,0,3


In [ ]:
# 하루를 10분 단위의 144개로 나누어, 충전 시작 시간에 맞게 할당 (y 구해야 함)
def time_calculate(time):
    hour, minute = time.split()[1].split(':')
    total_minutes = int(hour) * 60 + int(minute)
    time_interval = total_minutes // 10 
    return time_interval

df['time'] = df['start_time'].apply(time_calculate)